# Physics-enhanced NeuralODEs in real-world applications
Tutorial by Tobias Thummerer, Johannes Stoljar
*(Work in progress, last edit: 09.12.2022)*

## Keywords
PeNODE, NeuralODE, Universal Differential Equation, Hybrid Modeling, Functional Mock-up Unit, FMU, NeuralFMU

## License

In [1]:
# Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMIFlux.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
The Julia Package *FMIFlux.jl* is motivated by the application of hybrid modeling. This package enables the user to integrate his simulation model(s) in form of FMU(s) between neural networks (NeuralFMU). For more detailed information on *FMIFlux.jl*, see the [introduction page](https://thummeto.github.io/FMIFlux.jl/dev/). This tutorial is an easy, code-focussed version of the paper [[1]](#Source).

## Introduction to the example
In this example, a real-world simulation model is enhanced in terms of accuracy using a so called *physics-enhanced neural ordinary differential equation* (PeNode). Basically, this is an extension to the NeuralODE concept and looks as can be seen in Fig. 1.

![PeNODE.svg](https://github.com/thummeto/FMIFlux.jl/blob/main/docs/src/examples/pics/PeNODE.svg?raw=true)

*Fig.1: A possible representation for a physics-enhanced neural ordinary differential equation (PeNODE)*.

Note, that this is only one possible topology, there could be an additional artifical neural network (ANN) before the ODE (that can also share some connections with the other ANN) and other signals, like FMU state derivatives, inputs and outputs could be connected to the ANN(s). ODEs are in general not very handy, for modeling **real** applications, a more suitable format is needed. The most common model exchange format in industry is the *Functional Mock-up interface* (FMI), models exported with FMI are called *Functional Mock-up unit* (FMU). Especially *model-exchange* FMUs can be seen as containers for ODEs. For more information, see ([fmi-standard.org](http://fmi-standard.org/)). So if you want to use a real model from your modeling tool (with FMI support), you can simply export an FMU instead of handling large and bulky ODEs. If PeNODEs use FMUs instead of ODEs, they are called NeuralFMUs [[2]](#Source).

In this tutorial series, a *vehicle longitudinal-dynamics model* (VLDM) is extended to a NeuralFMU (PeNODE) to make better consumption predictions.


## Target group
The example is primarily intended for users who work in the field of first principle and/or hybrid modeling and are further interested in hybrid model building. The example wants to show how simple it is to combine FMUs with machine learning and to illustrate the advantages of this approach.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMIFlux.jl/blob/examples/examples/mdpi_part1_loading.ipynb) there is also a [Julia file](https://github.com/thummeto/FMIFlux.jl/blob/examples/examples/mdpi_part1_loading.jl) with the same name, which contains only the code cells. For the documentation there is a [Markdown file](https://github.com/thummeto/FMIFlux.jl/blob/examples/examples/mdpi_part1_loading.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | 
|:----|:----------------------------------|:--------------------------|
| 1.  | Enter Package Manager via         | ]                         |          
| 2.  | Install FMI via                   | add FMI                   | 
| 3.  | Install FMIFlux via               | add FMIFlux               |
| 4.  | Install FMIZoo via                | add FMIZoo                |
| 5.  | Install Plots via                 | add Plots                 |
| 6.  | Install PlotlyJS via              | add PlotlyJS              |
| 7.  | Install Random via                | add Random                | 
| 8.  | Install JLD2 via                  | add JLD2                  | 

## Part 1: Loading the FMU

To run the example, the previously installed packages must be included. 

In [2]:
# Loading in the required libraries
using FMIFlux   # for NeuralFMUs
using FMI       # import FMUs into Julia 
using FMIZoo    # a collection of demo models, including the VLDM

import FMI.DifferentialEquations: Tsit5     # import the Tsit5-solver
using FMIFlux.Flux                          # the default ML-library 
using JLD2                                  # data format for saving/loading parameters

# plotting
import Plots        # default plotting framework
import PlotlyJS     # plotting (interactive)
Plots.plotlyjs()    # actiavte PlotlyJS as default plotting backend

# Let's fix the random seed to make our program determinsitic (ANN layers are initialized indeterminsitic otherwise)
import Random 
Random.seed!(1234)

# we use the Tsit5 solver for ODEs here 
solver = Tsit5()    

Tsit5(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false))

Next, we load the FMU from the *FMIZoo.jl* and have a brief look on its metadata. For a detailed view, see the [Modelica model](https://github.com/ThummeTo/FMIZoo.jl/blob/main/models/src/VLDM.mo).

In [3]:
# load our FMU (we take one from the FMIZoo.jl, exported with Dymola 2022x)
fmu = fmiLoad("VLDM", "Dymola", "2022x"; type=:ME, logLevel=FMI.FMIImport.FMULogLevelInfo)

# let's have a look on the model meta data
fmiInfo(fmu)
# States:                         6
# 33554432 ["driver.accelerationPedalController.PI.x"]                                                          "PI-Controller state (integrated error), accelerating"
# 33554433 ["driver.brakePedalController.PI.x"]                                                                 "PI-Controller state (integrated error), braking"
# 33554434 ["drivingCycle.s"]                                                                                   "vehicle position (target)"
# 33554435 ["dynamics.accelerationCalculation.integrator.y"]                                                    "vehcle position (actual)"
# 33554436 ["dynamics.accelerationCalculation.limiter.u", "dynamics.accelerationCalculation.limIntegrator.y"]   "vehicle velocity (actual)"
# 33554437 ["result.integrator.y"]                                                                              "cumulative consumption * 3600"

┌ Info: fmi2Load(...): FMU resources location is `file:///C:/Users/thummeto/AppData/Local/Temp/fmijl_AhqPP0/VLDM/resources`
└ @ FMICore C:\Users\thummeto\Documents\FMICore.jl\src\logging.jl:20


#################### Begin information for FMU ####################
	Model name:			Longitudinaldynamic.LongitudinaldynamicmodelContinuous
	FMI-Version:			2.0
	GUID:				{669889ab-7ab7-4fac-be92-96b6cd0b86a6}
	Generation tool:		Dymola Version 2020x (64-bit), 2019-10-10
	Generation time:		2022-07-22T09:32:50Z
	Var. naming conv.:		structured
	Event indicators:		28
	Inputs:				0
	Outputs:			0
	States:				6
		33554432 ["driver.accelerationPedalController.PI.x"]
		33554433 ["driver.brakePedalController.PI.x"]
		33554434 ["drivingCycle.s"]
		33554435 ["dynamics.accelerationCalculation.integrator.y"]
		33554436 ["dynamics.accelerationCalculation.limiter.u", "dynamics.accelerationCalculation.limIntegrator.y", "dynamics.accelerationCalculation.limiter.simplifiedExpr"]
		33554437 ["result.integrator.y"]
	Supports Co-Simulation:		true
		Model identifier:	Longitudinaldynamic_LongitudinaldynamicmodelContinuous
		Get/Set State:		true
		Serialize State:	true
		Dir. Derivatives:	true
		Var. com. steps:	

Next thing is having a look on the real measurement data, that comes with the FMU. The VLDM and corresponding data are based on the [Component Library for Full Vehicle Simulations](https://github.com/TUMFTM/Component_Library_for_Full_Vehicle_Simulations) [[3]](#Source).

In [4]:
# load data from FMIZoo.jl, gather simulation parameters for FMU
data = FMIZoo.VLDM(split=:train)
tStart = data.consumption_t[1]
tStop = data.consumption_t[end]
tSave = data.consumption_t

# have a look on the FMU parameters
data.params

Dict{String, Any} with 3 entries:
  "peFileName" => "C:\\Users\\thummeto\\Documents\\FMIZoo.jl\\src\\..\\data\\VL…
  "edFileName" => "C:\\Users\\thummeto\\Documents\\FMIZoo.jl\\src\\..\\data\\VL…
  "dcFileName" => "C:\\Users\\thummeto\\Documents\\FMIZoo.jl\\src\\..\\data\\VL…

Finally, we do a single simulation run and compare the simulation output to the real data.

In [5]:

# let's run a simulation
resultFMU = fmu(t=(tStart, tStop), parameters=data.params) 
fig = fmiPlot(resultFMU)                                                                        # This is a bit too much, so ...
fig = fmiPlot(resultFMU; stateIndices=6:6)                                                      # ... only plot the state #6 and ...
fig = fmiPlot(resultFMU; stateIndices=6:6, ylabel="Cumulative consumption [Ws]", label="FMU")   # ... add some helpful labels!

# plot the (measurement) data 
Plots.plot!(fig, data.consumption_t, data.consumption_val; label="Data", ribbon=data.consumption_dev, fillalpha=0.3)

TypeError: TypeError: in keyword argument t, expected Union{Nothing, Real}, got a value of type Tuple{Float64, Float64}

**WIP, to be continued soon ...**

### Source

[1] Tobias Thummerer, Johannes Stoljar and Lars Mikelsons. 2022. **NeuralFMU: presenting a workflow for integrating hybrid NeuralODEs into real-world applications.** Electronics 11, 19, 3202. [DOI: 10.3390/electronics11193202](https://doi.org/10.3390/electronics11193202)


[2] Tobias Thummerer, Lars Mikelsons and Josef Kircher. 2021. **NeuralFMU: towards structural integration of FMUs into neural networks.** Martin Sjölund, Lena Buffoni, Adrian Pop and Lennart Ochel (Ed.). Proceedings of 14th Modelica Conference 2021, Linköping, Sweden, September 20-24, 2021. Linköping University Electronic Press, Linköping (Linköping Electronic Conference Proceedings ; 181), 297-306. [DOI: 10.3384/ecp21181297](https://doi.org/10.3384/ecp21181297)

[3] Danquah, B.; Koch, A.; Weis, T.; Lienkamp, M.; Pinnel, A. 2019. **Modular, Open Source Simulation Approach: Application to Design
and Analyze Electric Vehicles.** In Proceedings of the IEEE 2019 Fourteenth International Conference on Ecological Vehicles and
Renewable Energies (EVER), Monte Carlo, Monaco, 8–10 May 2019; pp. 1–8. [DOI: 10.1109/EVER.2019.8813568](https://doi.org/10.1109/EVER.2019.8813568).
